# импорты

In [1]:
import pandas as pd
import numpy as np
from jury import Jury
from bs4 import BeautifulSoup
from collections import Counter
from nltk import word_tokenize, ngrams

In [2]:
#stop_list = ' ,./?«":»()1234567890§—-™æ°+₽♦•■№{®£½”“=&©|„º}–⅟…$†‡́’*¬~<>_‘][^'
stop_list = list('¬ ')

In [3]:
def line_signs(line):
    return line.replace('\n', '').replace('\t', '').replace('\u2003', '')

In [4]:
# read original file
with open('new', 'r', encoding='utf-8') as f:
    soup_raw = [line_signs(f.read())]

In [6]:
# read clean file
with open('volume_32.xml', 'r', encoding='utf-8') as f:
    soup_cleaned = [line_signs(BeautifulSoup(f, 'xml').get_text())]

в качестве чистого файла у нас 32 том из папки дата, в качестве базового - самый-самый первый распознанный файл, который еще в документе майкрософта живет.

In [8]:
# character count
c_raw = dict(Counter(soup_raw[0].lower()))
c_cleaned = dict(Counter(soup_cleaned[0].lower()))

мы подсчитали, сколько и каких у нас символов встретилось в каждом документе. Если бы распознавание работало идеально, эти числа были бы одинаковы для всех символов, ведь мы имеем дело с одним и тем же документом. Если распознавание работает не идеально, но никакие конкретные символы не страдают особенно часто, то разница между числом всех символов в двух документов не выйдет за пределы стандартного отклонения. Сейчас мы будем пытаться найти символы, для которых эта разница все-таки выходит за стандартное отклонение, держа в голове док с частыми ошибками, встреченными разметчиками.

для удобства работы сведем полученные количества в один датафрейм

In [9]:
frame_c_raw = pd.DataFrame.from_dict(c_raw, orient='index')
frame_c_cleaned = pd.DataFrame.from_dict(c_cleaned, orient='index')

In [10]:
frame_c = pd.concat([frame_c_raw, frame_c_cleaned], axis=1)

In [ ]:
frame_c.columns = ['raw', 'clean']
frame_c = frame_c.fillna(0)
frame_c_compared = frame_c.drop(stop_list)
frame_c_compared

выкидываем отсюда пробелы и символы переноса - они нам, кажется, ничего особо нового не скажут, - и находим нормализованную разницу.

In [12]:
frame_c_compared['dif'] = (frame_c_compared.raw - frame_c_compared.clean)/151

In [13]:
frame_c_compared[60:80]

,raw,clean,dif
ѣ,32674.0,37268.0,-30.423841
!,1877.0,1980.0,-0.682119
х,16085.0,18510.0,-16.059603
ю,14023.0,15500.0,-9.781457
э,6025.0,6785.0,-5.033113
:,2620.0,2932.0,-2.066225
x,501.0,588.0,-0.576159
—,4586.0,4953.0,-2.430464
»,1602.0,1821.0,-1.450331
«,4154.0,3667.0,3.225166


запросим среднее значение и стандартное отклонение и выясним, где находятся границы, где разница сильно отклоняется от средней

In [14]:
frame_c_compared.describe()

,raw,clean,dif
count,151.000000,151.000000,151.000000
mean,12646.953642,14288.476821,-10.871014
std,29686.347237,33541.084742,27.093486
min,0.000000,0.000000,-144.966887
25%,7.000000,6.500000,-4.811258
50%,218.000000,461.000000,-0.622517
75%,3863.000000,3933.500000,0.013245
max,183473.000000,204709.000000,37.562914


In [15]:
lower_crit_c = -37.9645
higher_crit_c = 16.222472
frame_c_val = frame_c_compared[(frame_c_compared.dif > higher_crit_c) | (frame_c_compared.dif < lower_crit_c)]

теперь можно посмотреть, что у нас с разницей по символам

отрицательная разница значит, что символа больше в чистом документе; положительная разница - символа больше в необработанном. 

In [16]:
frame_c_val.sort_values('dif')

,raw,clean,dif
н,103728.0,125618.0,-144.966887
о,183473.0,204709.0,-140.635762
и,103280.0,121299.0,-119.331126
а,131200.0,147479.0,-107.807947
е,113418.0,128497.0,-99.860927
т,106515.0,120103.0,-89.986755
с,92287.0,104673.0,-82.026490
р,77937.0,89125.0,-74.092715
ъ,86140.0,97291.0,-73.847682
в,78494.0,87996.0,-62.927152


вещи, которые отмечали разметчики:
 - "н" и "и" часто заменяются на "п"
 - "и" заменяется на "н"
 - "iо" и "ю" заменяется на "р"
 - "ы" и "ь" путаются

так, найденные различия действительно в принципе соотносятся с тем, что отмечали разметчики - за исключением разве что ы и ь. Надо пройтись по биграммам из символов, чтобы отследить уже другие жалобы, на замены типа "их" - "нх"/"пх"

механизм примерно тот же - найти все сочетания двух символов по двум документам, залить в датафрейм, выделить разницу

In [18]:
def tuple_join(one_tuple):
    return str(one_tuple[0])+str(one_tuple[1])

In [19]:
raw_bigrams_char = [x for x in map(tuple_join, list(ngrams(soup_raw[0].lower(), 2))) if x[0] not in stop_list and x[1] not in stop_list]
#raw_bigrams_char = [x for x in map(tuple_join, list(ngrams(soup_raw[0].lower(), 2))) if x[0]!=' ' and x[1]!=' ']
cleaned_bigrams_char = [x for x in map(tuple_join, list(ngrams(soup_cleaned[0].lower(), 2))) if x[0] not in stop_list and x[1] not in stop_list]

In [20]:
raw_bigrams_char_count = dict(Counter(raw_bigrams_char))
cleaned_bigrams_char_count = dict(Counter(cleaned_bigrams_char))

In [331]:
raw_bigrams_char_frame = pd.DataFrame.from_dict(raw_bigrams_char_count, orient='index')
cleaned_bigrams_char_frame = pd.DataFrame.from_dict(cleaned_bigrams_char_count, orient='index')

In [ ]:
bigrams_char_c

In [333]:
bigrams_char_c = pd.concat([raw_bigrams_char_frame, cleaned_bigrams_char_frame], axis=1).fillna(0)

In [ ]:
bigrams_char_c.columns = ['raw', 'clean']
bigrams_char_c['dif'] = (bigrams_char_c.raw - bigrams_char_c.clean)/4549

In [336]:
bigrams_char_c.describe()

,raw,clean,dif
count,4549.000000,4549.000000,4549.000000
mean,349.814465,395.706089,-0.010088
std,1534.059207,1758.026792,0.054641
min,0.000000,0.000000,-1.054517
25%,1.000000,0.000000,-0.001319
50%,3.000000,2.000000,0.000000
75%,34.000000,37.000000,0.000220
max,26136.000000,29773.000000,0.397230


In [337]:
lower_crit = -0.064729
higher_crit = 0.044553
bigrams_char_val = bigrams_char_c[(bigrams_char_c.dif > higher_crit) | (bigrams_char_c.dif < lower_crit)]

на что жаловались разметчики:
- "на" прочитано как "па"
- "ни" прочитано как "пи"
- "он" прочитано как "оп"
- "их" прочитано как "нх"
- "не" прочитано как "пе"

биграмм у нас значительно больше, чем символов, так что весь список выходящих за рамки колебаний биграмм мы сразу взглядом не окинем. Вытащим список из пятнадцати биграмм, которые чаще встречаются в чистом документе, чем в грязном.

In [338]:
bigrams_char_val.sort_values('dif')[:15]

,raw,clean,dif
на,18048.0,22845.0,-1.054517
ст,26136.0,29773.0,-0.799516
но,17486.0,20826.0,-0.734227
не,11300.0,14422.0,-0.686305
..,2735.0,5766.0,-0.666300
он,7495.0,10469.0,-0.653770
ра,16594.0,19401.0,-0.617059
ен,16432.0,19178.0,-0.603649
ов,13881.0,16213.0,-0.512640
ос,12709.0,14840.0,-0.468455


и список из пятнадцати биграмм, которые больше нравятся грязному документу.

In [339]:
bigrams_char_val.sort_values('dif', ascending=False)[:15]

,raw,clean,dif
па,5201.0,3394.0,0.397230
іі,1370.0,36.0,0.293251
оп,4260.0,2935.0,0.291273
.—,1226.0,156.0,0.235217
пе,5225.0,4350.0,0.192350
пъ,987.0,142.0,0.185755
-с,1350.0,663.0,0.151022
-н,1097.0,531.0,0.124423
-т,1348.0,952.0,0.087052
-в,715.0,321.0,0.086612


что видим? в грязном - сочетания с сомнительными символами из знаков препинания, подозрительно много ii, в лидерах и правда "па" и другие сочетания с "п", как и ожидалось по словам разметчиков. в чистом - сочетания с буквами "н" и "и", собственно, и пострадавшими от количества "п"; из любопытного - две точки (многоточие)?

нужно учитывать, что даже проверенный разметчиками текст не идеален и содержит незамеченные ошибки

# дальше будет жить все, что еще не живет

оценка точности АББИИ без спеллчекера модулем Jury

In [347]:
scorer = Jury()

In [348]:
scores = scorer.evaluate(predictions=soup_raw, references=soup_cleaned)

In [350]:
dict(scores)

{'empty_predictions': 0,
 'total_items': 1,
 'bleu_1': {'score': 0.7821024009231794,
  'precisions': [0.8838556957744746],
  'brevity_penalty': 0.8848756699337278,
  'length_ratio': 0.8910209001013504,
  'translation_length': 325285,
  'reference_length': 365070},
 'bleu_2': {'score': 0.7281178234959058,
  'precisions': [0.8838556957744746, 0.7660505896385927],
  'brevity_penalty': 0.8848756699337278,
  'length_ratio': 0.8910209001013504,
  'translation_length': 325285,
  'reference_length': 365070},
 'bleu_3': {'score': 0.6792911507768497,
  'precisions': [0.8838556957744746, 0.7660505896385927, 0.6681627997774245],
  'brevity_penalty': 0.8848756699337278,
  'length_ratio': 0.8910209001013504,
  'translation_length': 325285,
  'reference_length': 365070},
 'bleu_4': {'score': 0.6343639340409344,
  'precisions': [0.8838556957744746,
   0.7660505896385927,
   0.6681627997774245,
   0.5838533949004249],
  'brevity_penalty': 0.8848756699337278,
  'length_ratio': 0.8910209001013504,
  'tra

надо разбираться, что он вообще имеет в виду, но данные красивые - можно прикрепить в виде таблички (с пояснениями, что это за метрики, конечно)

штуки, которые надо пробовать

queue: tesseract(tesseract-ocr-wrapper, https://github.com/Altabeh/tesseract-ocr-wrapper), easyOCR(https://github.com/JaidedAI/EasyOCR), Attention-OCR(https://github.com/da03/Attention-OCR), doctr(https://github.com/mindee/doctr), Ocular(https://github.com/tberg12/ocular)

апдейт 1: doctr попробован в облачной версии в huggingface во всех своих лицах, и все четыре модели споткнулись о невозможность угадать язык и выдавали кашу. Из плюсов? Была возможность потыкать в разные модели для детекции текста на изображении, и РезНеты определенно хороши и справляются с проступающими с другой страницы строками. Надо узнать, чем текст находит АББИИ и при необходимости иметь в виду какую-нибудь небольшую РезНет. Но сейчас буду пробовать просто дать ему новый словарь символов